# Training Status Process
> Process to handle training data stream

In [3]:
# | default_exp training_status_process

In [15]:
# | export

import random
from datetime import datetime, timedelta
from os import environ
from time import sleep
from typing import *

import asyncio
from asyncer import asyncify
from sqlalchemy.exc import NoResultFound
from sqlmodel import Session, select, func

import airt_service
from airt_service.data.clickhouse import get_count
from airt_service.db.models import get_session_with_context, User, TrainingStreamStatus
from airt.logger import get_logger
from airt.patching import patch

In [5]:
import contextlib
import threading
from pathlib import Path

import numpy as np
import pandas as pd
import pytest
import uvicorn
from confluent_kafka import Producer, Consumer
from _pytest.monkeypatch import MonkeyPatch

from airt_service.confluent import confluent_kafka_config, create_topics_for_user
from airt_service.db.models import create_user_for_testing
from airt_service.helpers import set_env_variable_context
from airt_service.server import create_ws_server
from airt_service.sanitizer import sanitized_print

23-01-11 05:33:31.151 [INFO] airt.executor.subcommand: Module loaded.


In [6]:
test_username = create_user_for_testing()
display(test_username)

'yuqnsygtsr'

In [7]:
# | exporti

logger = get_logger(__name__)

In [9]:
with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == test_username)).one()
    test_start_event = TrainingStreamStatus(account_id=999, event="start", count=0, user=user)
    test_end_event = TrainingStreamStatus(account_id=999, event="end", count=10000, user=user)
    session.add(test_start_event)
    session.commit()
    session.add(test_end_event)
    session.commit()
    
    test_start_event = TrainingStreamStatus(account_id=666, event="start", count=0, user=user)
    
    session.add(test_start_event)
    session.commit()


In [36]:
with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == test_username)).one()
    #     select(TrainingStreamStatus.account_id).distinct()
    #     events = session.exec(
    #         select(TrainingStreamStatus)
    #         .where(TrainingStreamStatus.user == user)
    #         .order_by(TrainingStreamStatus.created.desc())
    #         .order_by(TrainingStreamStatus.id.desc())
    #         .group_by(TrainingStreamStatus.account_id)
    #     )
    events = session.exec(
        select(TrainingStreamStatus).distinct(TrainingStreamStatus.account_id).order_by(TrainingStreamStatus.id.desc())
    )
    for event in events:
        display(event)

TrainingStreamStatus(uuid=UUID('c022a771-74f5-40a8-b498-ed55c2669044'), id=3, count=0, user_id=134, account_id=666, event=<TrainingEvent.start: 'start'>, created=datetime.datetime(2023, 1, 11, 5, 35, 23))

TrainingStreamStatus(uuid=UUID('dcf46385-15be-42ce-8db1-1790bd956c11'), id=2, count=10000, user_id=134, account_id=999, event=<TrainingEvent.end: 'end'>, created=datetime.datetime(2023, 1, 11, 5, 35, 23))

TrainingStreamStatus(uuid=UUID('da50976f-6229-4369-b77e-f254838b0445'), id=1, count=0, user_id=134, account_id=999, event=<TrainingEvent.start: 'start'>, created=datetime.datetime(2023, 1, 11, 5, 35, 23))

In [22]:
help(select)

Help on function select in module sqlmodel.sql.expression:

select(*entities: Any, **kw: Any) -> Union[sqlmodel.sql.expression.Select, sqlmodel.sql.expression.SelectOfScalar]



In [25]:
import sqlmodel
dir(sqlmodel.sql.expression.SelectOfScalar)

['__bool__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__visit_name__',
 '__weakref__',
 '_add_context_option',
 '_all_selected_columns',
 '_annotate',
 '_annotations',
 '_annotations_cache_key',
 '_assert_no_memoizations',
 '_attributes',
 '_auto_correlate',
 '_bind',
 '_cache_key_traversal',
 '_clone',
 '_clone_annotations_traverse_internals',
 '_cloned_set',
 '_compile_options',
 '_compile_state_factory',
 '_compile_state_plugin',
 '_compile_w_cache',
 '_compiler',
 '_compiler_dispatch',
 '_constructor',
 '_copy_internals',
 '_correlate',
 '_correlate_exc

In [6]:
# | export


def get_recent_event_for_user(username: str) -> Optional[TrainingStreamStatus]:
    with get_session_with_context() as session:
        user = session.exec(select(User).where(User.username == username)).one()
        try:
            event = session.exec(
                select(TrainingStreamStatus)
                .where(TrainingStreamStatus.user == user)
                .order_by(TrainingStreamStatus.created.desc())  # type: ignore
                .order_by(TrainingStreamStatus.id.desc())  # type: ignore
                .limit(1)
            ).one()
        except NoResultFound:
            return None
    return event

In [7]:
end_count = 1_000_000

with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == test_username)).one()
    actual = get_recent_event_for_user(username=test_username)
    assert actual == None, actual
    test_start_event = TrainingStreamStatus(event="start", count=0, user=user)
    test_end_event = TrainingStreamStatus(event="end", count=end_count, user=user)
    session.add(test_start_event)
    session.commit()
    session.add(test_end_event)
    session.commit()

    actual = get_recent_event_for_user(username=test_username)
    display(actual)
    assert actual.event == "end", actual
    assert actual.count == end_count, actual
    assert actual.user_id == user.id, actual

TrainingStreamStatus(event=<TrainingEvent.end: 'end'>, count=1000000, user_id=29, uuid=UUID('1a713ab7-1d7a-48d1-a3c4-2b9adbe26320'), id=174, created=datetime.datetime(2023, 1, 10, 16, 0, 56))

In [8]:
# | export


def get_count_from_training_data_ch_table() -> int:
    return airt_service.data.clickhouse.get_count(
        username=environ["KAFKA_CH_USERNAME"],
        password=environ["KAFKA_CH_PASSWORD"],
        host=environ["KAFKA_CH_HOST"],
        port=int(environ["KAFKA_CH_PORT"]),
        database=environ["KAFKA_CH_DATABASE"],
        table=environ["KAFKA_CH_TABLE"],
        protocol=environ["KAFKA_CH_PROTOCOL"],
    )

In [9]:
with MonkeyPatch.context() as monkeypatch:
    monkeypatch.setattr(
        "__main__.get_count_from_training_data_ch_table",
        lambda: 999,
    )
    #     monkeypatch.setattr(
    #         "airt_service.data.clickhouse.get_count",
    #         lambda: 999,
    #     )
    actual = get_count_from_training_data_ch_table()
    display(actual)
    assert actual == 999, actual

999

In [10]:
# | export


@patch(cls_method=True)
def _create(
    cls: TrainingStreamStatus, *, event: str, count: int, user: User, session: Session
) -> TrainingStreamStatus:
    training_event = TrainingStreamStatus(event=event, count=count, user=user)
    session.add(training_event)
    session.commit()
    return training_event

In [11]:
# | export


async def process_training_status(username: str):
    # Get recent event for username
    prev_count = 0
#     prev_check_on = datetime.utcnow()
    while True:
        recent_event = await asyncify(get_recent_event_for_user)(username)
        logger.info(f"{recent_event=}")
        if recent_event is None:
            pass
        elif recent_event.event == "end":
            # Check model training status started and start it if not already
            pass
        elif recent_event.event in ["start", "upload"]:
            curr_count = await asyncify(get_count_from_training_data_ch_table)()
            curr_check_on = datetime.utcnow()

            with get_session_with_context() as session:
                user = session.exec(select(User).where(User.username == username)).one()
                if (
                    curr_count == prev_count
                    and curr_check_on - recent_event.created > timedelta(seconds=10)
                ):
                    end_event = await asyncify(TrainingStreamStatus._create)(
                        event="end", count=curr_count, user=user, session=session
                    )
                    prev_count = 0
                    # Start model training status
                elif curr_count != prev_count:
                    upload_event = await asyncify(TrainingStreamStatus._create)(
                        event="upload", count=curr_count, user=user, session=session
                    )
                    prev_count = curr_count
#                 prev_check_on = curr_check_on

        sleep(random.randint(1, 2))

In [12]:
definitions = [
    "appLaunch",
    "sign_in",
    "sign_out",
    "add_to_cart",
    "purchase",
    "custom_event_1",
    "custom_event_2",
    "custom_event_3",
]


applications = ["DriverApp", "PUBG", "COD"]


def generate_n_rows_for_training_data(n: int, seed: int = 42):
    rng = np.random.default_rng(seed=seed)
    account_id = rng.choice([4000, 5000, 500], size=n)
    definition_id = rng.choice(definitions, size=n)
    application = rng.choice(applications, size=n)
    occurred_time_ticks = rng.integers(
        datetime(year=2022, month=1, day=1).timestamp() * 1000,
        datetime(year=2022, month=11, day=1).timestamp() * 1000,
        size=n,
    )
    occurred_time = pd.to_datetime(occurred_time_ticks, unit="ms").strftime(
        "%Y-%m-%dT%H:%M:%S.%f"
    )
    person_id = rng.integers(n // 10, size=n)

    df = pd.DataFrame(
        {
            "AccountId": account_id,
            "Application": application,
            "DefinitionId": definition_id,
            "OccurredTimeTicks": occurred_time_ticks,
            "OccurredTime": occurred_time,
            "PersonId": person_id,
        }
    )
    return json.loads(df.to_json(orient="records"))


generate_n_rows_for_training_data(100)[-1]

{'AccountId': 4000,
 'Application': 'DriverApp',
 'DefinitionId': 'custom_event_3',
 'OccurredTimeTicks': 1652181248527,
 'OccurredTime': '2022-05-10T11:14:08.527000',
 'PersonId': 6}

In [13]:
class Server(uvicorn.Server):
    def install_signal_handlers(self):
        pass

    @contextlib.contextmanager
    def run_in_thread(self):
        thread = threading.Thread(target=self.run)
        thread.start()
        try:
            while not self.started:
                sleep(1e-3)
            yield
        finally:
            self.should_exit = True
            thread.join()


def delivery_report(err, msg):
    """Called once for each message produced to indicate delivery result.
    Triggered by poll() or flush()."""
    if err is not None:
        sanitized_print("Message delivery failed: {}".format(err))
    else:
        #         sanitized_print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))
        pass

In [14]:
create_topics_for_user(username=test_username)


def test_process_training_status():
    logger.info("I am done at tests")
    with get_session_with_context() as session:
        user = session.exec(select(User).where(User.username == test_username)).one()
        test_start_event = TrainingStreamStatus(event="start", count=0, user=user)
        session.add(test_start_event)
        session.commit()

        p = Producer(confluent_kafka_config)
        msg_count = 1000
        training_data = generate_n_rows_for_training_data(msg_count, seed=999)
        for i in range(msg_count):
            p.produce(
                f"{test_username}_training_data",
                json.dumps(training_data[i]).encode("utf-8"),
                on_delivery=delivery_report,
            )
        p.flush()

        while True:
            recent_event = get_recent_event_for_user(test_username)
            sleep(5)
            display(f"in tests - {recent_event=}")
            if recent_event.event == "end":
                break


with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
    with MonkeyPatch.context() as monkeypatch:
        monkeypatch.setattr(
            "__main__.get_count_from_training_data_ch_table",
            lambda: 999,
        )
        app, fast_kafka_api_app = create_ws_server(assets_path=Path("../assets"))

        @fast_kafka_api_app.run_in_background()
        async def startup_event():
            await process_training_status(username=test_username)

        #         while True:
        #             logger.info("I am still running")
        #             await asyncio.sleep(1)
        #             sleep(1)
        #         process_training_status(username=test_username)

        config = uvicorn.Config(app, host="127.0.0.1", port=6006, log_level="debug")
        server = Server(config=config)

        with server.run_in_thread():
            # Server started.
            sanitized_print("server started")
            #         sleep(5)
            test_process_training_status()

        sanitized_print("server stopped")
        # Server stopped.

23-01-10 16:00:55.807 [INFO] airt_service.confluent: Topic wgmqzkwigw_training_data created
23-01-10 16:00:55.808 [INFO] airt_service.confluent: Topic wgmqzkwigw_realitime_data created
23-01-10 16:00:55.809 [INFO] airt_service.confluent: Topic wgmqzkwigw_training_data_status created
23-01-10 16:00:55.810 [INFO] airt_service.confluent: Topic wgmqzkwigw_training_model_status created
23-01-10 16:00:55.810 [INFO] airt_service.confluent: Topic wgmqzkwigw_model_metrics created
23-01-10 16:00:55.810 [INFO] airt_service.confluent: Topic wgmqzkwigw_prediction created
23-01-10 16:00:55.936 [INFO] airt_service.server: kafka_config={'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092', 'group_id': 'kumaran-airt-service-kafka-1:9092_group', 'auto_offset_reset': 'earliest'}


%4|1673366455.735|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1673366455.735|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
INFO:     Started server process [10966]
INFO:     Waiting for application startup.


23-01-10 16:00:56.017 [INFO] fast_kafka_api._components.asyncapi: Keeping the old async specifications at: 'asyncapi/spec/asyncapi.yml'
23-01-10 16:00:56.018 [INFO] fast_kafka_api._components.asyncapi: Skipping generating async documentation in '/work/airt-service/notebooks/asyncapi/docs'
23-01-10 16:00:56.066 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting..
23-01-10 16:00:56.067 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created.
23-01-10 16:00:56.067 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting..
23-01-10 16:00:56.068 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:6006 (Press CTRL+C to quit)


server started
23-01-10 16:00:56.071 [INFO] __main__: I am done at tests
23-01-10 16:00:56.094 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
23-01-10 16:00:56.095 [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'infobip_training_data'})
23-01-10 16:00:56.096 [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'infobip_training_data'}
23-01-10 16:00:56.101 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
23-01-10 16:00:56.102 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
23-01-10 16:00:56.103 [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'infobip_realtime_data'})
23-01-10 16:00:56.103 [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'infobip_realtime_data'}
23-01-10 16:00:56.115 [INFO] fast_kafka_api._components.aiokafka_con

...

23-01-10 16:01:18.509 [INFO] __main__: recent_event=TrainingStreamStatus(event=<TrainingEvent.end: 'end'>, count=999, user_id=29, uuid=UUID('317f67e3-070a-422c-a8b2-1f468c4c50d9'), id=177, created=datetime.datetime(2023, 1, 10, 16, 1, 7))


INFO:     Waiting for application shutdown.


23-01-10 16:01:20.538 [INFO] __main__: recent_event=TrainingStreamStatus(event=<TrainingEvent.end: 'end'>, count=999, user_id=29, uuid=UUID('317f67e3-070a-422c-a8b2-1f468c4c50d9'), id=177, created=datetime.datetime(2023, 1, 10, 16, 1, 7))
23-01-10 16:01:21.543 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-01-10 16:01:21.544 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
23-01-10 16:01:21.544 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-01-10 16:01:21.545 [INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.


INFO:     Application shutdown complete.
INFO:     Finished server process [10966]


server stopped
